In [1]:
from torchtext.data import Dataset
from torchtext.data import Field, BucketIterator
import torchtext

In [2]:
SRC = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(init_token = '<sos>',
            eos_token = '<eos>')

In [3]:
train_data, val_data, test_data = torchtext.datasets.TranslationDataset.splits(
                            path = '',
                            exts=('.en', '.fa'),
                            train = 'train',
                            test = 'valid',

                            fields = (SRC,TRG)
)


In [4]:

# SRC.build_vocab(train_data, max_size=20000, min_freq = 3)
# TRG.build_vocab(train_data, max_size=20000, min_freq = 3)
SRC.build_vocab(train_data, max_size=25000, min_freq = 2)
TRG.build_vocab(train_data, max_size=25000, min_freq = 2)
# SRC.build_vocab(train_data, min_freq = 2)
# TRG.build_vocab(train_data, min_freq = 2)

In [5]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, val_data, test_data),
    sort_key = lambda x: len(x.src),
    sort_within_batch = True,
    batch_size = BATCH_SIZE,
    device = device)

In [6]:
# torch.save(dataloader_obj, 'dataloader.pth')

In [7]:
# torch.set_printoptions(edgeitems=3)
# a=next(iter(train_iterator))
# a.src

In [8]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ATTN_DIM = 64
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# ENC_EMB_DIM = 32
# DEC_EMB_DIM = 32
# ENC_HID_DIM = 64
# DEC_HID_DIM = 64
# ATTN_DIM = 8
########
# ENC_EMB_DIM = 64
# DEC_EMB_DIM = 64
# ENC_HID_DIM = 128
# DEC_HID_DIM = 128
# ATTN_DIM = 16
# ENC_EMB_DIM = 128
# DEC_EMB_DIM = 128
# ENC_HID_DIM = 256
# DEC_HID_DIM = 256
# ATTN_DIM = 32
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')



The model has 62,725,612 trainable parameters


In [9]:

PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)



In [10]:

import math
import time


def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [11]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')



Epoch: 01 | Time: 10m 51s
	Train Loss: 5.839 | Train PPL: 343.518
	 Val. Loss: 5.812 |  Val. PPL: 334.398
Epoch: 02 | Time: 10m 46s
	Train Loss: 5.066 | Train PPL: 158.480
	 Val. Loss: 5.613 |  Val. PPL: 273.836
Epoch: 03 | Time: 10m 53s
	Train Loss: 4.581 | Train PPL:  97.650
	 Val. Loss: 5.624 |  Val. PPL: 277.109
Epoch: 04 | Time: 10m 48s
	Train Loss: 4.199 | Train PPL:  66.642
	 Val. Loss: 5.683 |  Val. PPL: 293.884
Epoch: 05 | Time: 10m 50s
	Train Loss: 3.879 | Train PPL:  48.385
	 Val. Loss: 5.773 |  Val. PPL: 321.654
| Test Loss: 5.828 | Test PPL: 339.527 |


In [20]:
N_EPOCHS = 12
CLIP = 1

#best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 4m 52s
	Train Loss: 5.153 | Train PPL: 172.985
	 Val. Loss: 5.780 |  Val. PPL: 323.598
Epoch: 02 | Time: 4m 46s
	Train Loss: 4.974 | Train PPL: 144.632
	 Val. Loss: 5.747 |  Val. PPL: 313.233
Epoch: 03 | Time: 4m 45s
	Train Loss: 4.825 | Train PPL: 124.524
	 Val. Loss: 5.755 |  Val. PPL: 315.741
Epoch: 04 | Time: 4m 45s
	Train Loss: 4.693 | Train PPL: 109.141
	 Val. Loss: 5.769 |  Val. PPL: 320.360
Epoch: 05 | Time: 4m 47s
	Train Loss: 4.580 | Train PPL:  97.522
	 Val. Loss: 5.776 |  Val. PPL: 322.612
Epoch: 06 | Time: 4m 46s
	Train Loss: 4.485 | Train PPL:  88.676
	 Val. Loss: 5.765 |  Val. PPL: 319.057
Epoch: 07 | Time: 4m 47s
	Train Loss: 4.402 | Train PPL:  81.652
	 Val. Loss: 5.790 |  Val. PPL: 327.050
Epoch: 08 | Time: 4m 47s
	Train Loss: 4.325 | Train PPL:  75.565
	 Val. Loss: 5.832 |  Val. PPL: 340.901
Epoch: 09 | Time: 4m 49s
	Train Loss: 4.266 | Train PPL:  71.203
	 Val. Loss: 5.825 |  Val. PPL: 338.824
Epoch: 10 | Time: 4m 49s
	Train Loss: 4.205 | Train PPL

In [34]:
torch.save(model.state_dict(), 'tut256-model_ppl48.pt')

In [12]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)



    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]



    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, encoder_outputs)



        pred_token = output.argmax(1).item()

        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:]




In [13]:
example_idx = 16

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['especially', 'i', 'am', 'most', 'sorry', 'that', 'i', 'have', 'not', 'told', 'what', 'the', 'non', 'commissioned', 'of', 'us', 'did', '.']
trg = ['و', 'من', 'مخصوصا', 'بسیار', 'متاسفم', 'که', '،', 'آنچه', 'را', 'که', 'ارتشیان', 'گفتند', 'به', 'عنوان', 'یک', 'هم\u200cردیف', '.', 'نگفته\u200cام', '.', 'آن', 'ها', 'دقیق', 'و', 'روشن', 'نبودند', '.']


In [24]:
src1 = ['i', 'am', 'sorry']
translation= translate_sentence(src1, SRC, TRG, model, device)

print(f'predicted trg = {translation}')


predicted trg = ['خیلی', 'متاسفم', '.', '<eos>']


In [9]:
model.load_state_dict(torch.load('tut34-model_ppl60.pt', map_location=device))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.951 | Test PPL: 384.066 |


In [ ]:
model.load_state_dict(torch.load('tut34-model.pt', map_location=device))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')